# Notebook 5: Composing Datasets

In this notebook we will examine how we can use the elements we have encountered so far, in order to construct a TensorFlow dataset which will allow us to train machine learning models with data generated in real time. This is the core and most usefull functionality of the dataset portion of GravyFlow.

As usual, we begin by performing the relevent imports.

In [1]:
# Built-in imports
from typing import List, Dict
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten, Dropout, ELU
from tensorflow.keras.models import Model
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

2024-02-29 10:45:23.060816: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 10:45:23.060858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 10:45:23.062061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 10:45:23.069452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 10:45:24.478182: W tensorflow/compiler/tf2

In [2]:
env = gf.env()

INFO:root:TensorFlow version: 2.15.0, CUDA version: 12.2
2024-02-29 10:45:47.509205: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-02-29 10:45:47.509473: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2000 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
INFO:root:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def adjust_features(features, labels):
    labels['INJECTION_MASKS'] = labels['INJECTION_MASKS'][0]
    return features, labels

In [4]:
def create_gabbard_model(
        input_shape_onsource : int, 
        input_shape_offsource : int
    ) -> tf.keras.Model:
    
    # Define the inputs based on the dictionary keys and expected shapes
    # Replace `input_shape_onsource` and `input_shape_offsource` with the actual shapes
    onsource_input = Input(shape=input_shape_onsource, name="ONSOURCE")
    offsource_input = Input(shape=input_shape_offsource, name="OFFSOURCE")

    # Pass the inputs to your custom Whiten layer
    # Assuming your Whiten layer can handle multiple inputs
    whiten_output = gf.Whiten()([onsource_input, offsource_input])

    x = gf.Reshape()(whiten_output)
    
    # Convolutional and Pooling layers
    x = Conv1D(8, 64, padding='same', name="Convolutional_1")(x)
    x = ELU(name="ELU_1")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_1", padding="same")(x)
    
    x = Conv1D(8, 32, padding='same', name="Convolutional_2")(x)
    x = ELU(name="ELU_2")(x)
    x = Conv1D(16, 32, padding='same', name="Convolutional_3")(x)
    x = ELU(name="ELU_3")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_3", padding="same")(x)

    x = Conv1D(16, 16, padding='same', name="Convolutional_4")(x)
    x = ELU(name="ELU_4")(x)
    x = Conv1D(32, 16, padding='same', name="Convolutional_5")(x)
    x = ELU(name="ELU_5")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_5", padding="same")(x)
    
    x = Conv1D(32, 16, padding='same', name="Convolutional_6")(x)
    x = ELU(name="ELU_6")(x)
    
    # Flatten layer
    x = Flatten(name="Flatten")(x)
    
    # Dense layers with dropout
    x = Dense(64, name="Dense_1")(x)
    x = ELU(name="ELU_7")(x)
    x = Dropout(0.5, name="Dropout_1")(x)
    
    x = Dense(64, name="Dense_2")(x)
    x = ELU(name="ELU_8")(x)
    x = Dropout(0.5, name="Dropout_2")(x)
    
    outputs = Dense(1, activation='sigmoid', name="INJECTION_MASKS")(x)
    
    # Create model
    model = Model(inputs=[onsource_input, offsource_input], outputs=outputs, name="custom_model")
    
    return model

In [5]:
with env:
    # This object will be used to obtain real interferometer data based on specified parameters.
    ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
        data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
        data_labels=[                      # Define the types of data to include.
            gf.DataLabel.NOISE, 
            gf.DataLabel.GLITCHES
        ],
        segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
        force_acquisition=True,               # Force the acquisition of new data.
        cache_segments=False                  # Choose not to cache the segments.
    )

    # Initialize the noise generator wrapper:
    # This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
    noise: gf.NoiseObtainer = gf.NoiseObtainer(
        ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
        noise_type=gf.NoiseType.REAL,        # Specify the type of noise as REAL.
        ifos=gf.IFO.L1                       # Specify the interferometer (e.g., LIGO Livingston L1).
    )

    scaling_method : gf.ScalingMethod = gf.ScalingMethod(
        value=gf.Distribution(
            min_=8.0,
            max_=15.0,
            type_=gf.DistributionType.UNIFORM
        ),
        type_=gf.ScalingTypes.SNR
    )

    # Define a uniform distribution for the mass of the first object in solar masses.
    mass_1_distribution_msun : gf.Distribution = gf.Distribution(
        min_=10.0, 
        max_=60.0, 
        type_=gf.DistributionType.UNIFORM
    )

    # Define a uniform distribution for the mass of the second object in solar masses.
    mass_2_distribution_msun : gf.Distribution = gf.Distribution(
        min_=10.0, 
        max_=60.0, 
        type_=gf.DistributionType.UNIFORM
    )

    # Define a uniform distribution for the inclination of the binary system in radians.
    inclination_distribution_radians : gf.Distribution = gf.Distribution(
        min_=0.0, 
        max_=np.pi, 
        type_=gf.DistributionType.UNIFORM
    )

    # Initialize a PhenomD waveform generator with the defined distributions.
    # This generator will produce waveforms with randomly varied masses and inclination angles.
    phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
        mass_1_msun=mass_1_distribution_msun,
        mass_2_msun=mass_2_distribution_msun,
        inclination_radians=inclination_distribution_radians,
        scaling_method=scaling_method,
        injection_chance=0.5 # Set so half produced examples will not contain this signal
    )
    
    dataset : tf.data.Dataset = gf.Dataset(       
        noise_obtainer=noise,
        waveform_generators=phenom_d_generator,
        input_variables=[
            gf.ReturnVariables.ONSOURCE, 
            gf.ReturnVariables.OFFSOURCE, 
        ],
        output_variables=[
            gf.ReturnVariables.INJECTION_MASKS
        ]
    ).map(adjust_features)
    
    for input_example, _ in dataset.take(1):
        input_shape_onsource = input_example["ONSOURCE"].shape[1:]  # Exclude batch dimension    
        input_shape_offsource = input_example["OFFSOURCE"].shape[1:] 

    model = create_gabbard_model(input_shape_onsource, input_shape_offsource)

    # Now you can print the model summary
    model.summary()
    
    # Model compilation
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # Or any other loss function appropriate for your task
        metrics=['accuracy']
    )

2024-02-29 10:45:48.424549: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa8e4034b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-29 10:45:48.424576: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-02-29 10:45:48.429837: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-29 10:45:48.472450: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-29 10:45:48.527117: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
I0000 00:00:1709232348.587994  834282 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/home/michael.norman/miniconda3/envs/gravyflow/lib

Model: "custom_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 ONSOURCE (InputLayer)       [(None, 1, 4096)]            0         []                            
                                                                                                  
 OFFSOURCE (InputLayer)      [(None, 1, 32768)]           0         []                            
                                                                                                  
 whiten (Whiten)             (None, 1, 2048)              0         ['ONSOURCE[0][0]',            
                                                                     'OFFSOURCE[0][0]']           
                                                                                                  
 reshape (Reshape)           (None, 2048, 1)              0         ['whiten[0][0]']   

In [7]:
with env: 
    history = model.fit(
        dataset,
        epochs=10,  # Number of epochs to train for
        steps_per_epoch=100 /gf.Defaults.num_examples_per_batch
        #validation_data=validation_dataset  # Assuming you have a validation dataset
        # Add other parameters as needed, such as callbacks
    )

Epoch 1/10


/home/michael.norman/miniconda3/envs/gravyflow/lib/python3.10/site-packages/igwn_auth_utils/x509.py:98: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  return (cert.not_valid_after - datetime.utcnow()).total_seconds()
2024-02-29 10:46:12.075722: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


100/100 [==============================] - 15s 22ms/step - loss: 0.2001 - accuracy: 0.9200
Epoch 2/10
100/100 [==============================] - 6s 65ms/step - loss: 0.0582 - accuracy: 0.9822
Epoch 3/10
100/100 [==============================] - 9s 86ms/step - loss: 0.0720 - accuracy: 0.9841
Epoch 4/10
100/100 [==============================] - 11s 108ms/step - loss: 0.0834 - accuracy: 0.9750
Epoch 5/10
100/100 [==============================] - 8s 76ms/step - loss: 0.0525 - accuracy: 0.9853
Epoch 6/10
100/100 [==============================] - 5s 51ms/step - loss: 0.0226 - accuracy: 0.9941
Epoch 7/10
100/100 [==============================] - 11s 112ms/step - loss: 0.0535 - accuracy: 0.9866
Epoch 8/10
100/100 [==============================] - 6s 59ms/step - loss: 0.0399 - accuracy: 0.9894
Epoch 9/10
100/100 [==============================] - 8s 82ms/step - loss: 0.0532 - accuracy: 0.9866
Epoch 10/10
100/100 [==============================] - 8s 81ms/step - loss: 0.0436 - accuracy: 0.